



# Simulateur SVOM

Dans le cadre de la mission SVOM (Space Variable Objects Monitor) de détection de sursauts gamma plusieurs détecteurs, caméras et téléscopes ont été choisis pour des observations sur terre. Grâce à ce simulateur et ce tutoriel vous avez accès à des simulations de ce que peuvent nous donner ces différents instruments lors de phases de test (champ sombre, champ plat, ...) ou lors d'observations (données collectées grâce à différentes mesures faites sur les téléscopes).

## Sommaire :
1. Fonctionnement
    1. Fichiers de configuration
    2. Explication du code principal
2. Simulation d'une image    
3. Simulation d'une rampe
    1. Simulation
    2. Test des différents effets simulés
4. Simulation d'un bruit CDS

A réaliser au départ absolument :

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
%matplotlib notebook
import os
import numpy as np
import matplotlib.pyplot as plt
from ImSimpy.ImSimpy import ImageSimulator

# 1. Fonctionnement

### A. Fichiers de configuration

Dans le dossier *ImSimpy/ImSimpy/configFiles* et *pyETC/pyETC/telescope_database* se trouve des fichiers *.hjson* qui sont des fichiers de configuration. Vous pourrez par la suite choisir de simuler une rampe ou un bruit CDS (cf. 3ème partie ou 4ème partie respectivement).

Dans le dossier *ImSimpy/ImSimpy/configFiles* vous pouvez choisir de simuler des champs sombre (*Dark.hjson*), des champs plat (*Flat.hjson*), des images réalisées sur un temps très court pour mesurer le bruit de lecture (Texp=0s, *Bias.hjson*). Vous pouvez aussi décider de créer votre propre fichier .hjson en remplissant vous même le fichier *Make_one_image_is.hjson* (is pour image simulator). Dans ce fichier vous pouvez choisir le temps d'exposition, des paramètres de calcul, choisir des conditions locale comme la température ou l'élévation de la lune et des paramètres du téléscope comme le seeing ou des paramètres de caméras. Mais c'est surtout à l'intérieur de ce fichier que vous pouvez choisir d'observer le ciel et l'endroit d'observation.

Le fichier pris par défault est un Dark :  [Dark.hjson](../ImSimpy/configFiles/Dark.hjson)

*Remarque :* C'est à l'intérieur de ces fichiers que l'on choisit l'emplacement des fichiers .fits de configuration : carte des facteurs de conversion, carte des coefficients dépendant du temps du bruit thermique que produit les instruments devant le détecteur, carte des pixels morts et chauds dans le cas d'une simulation d'image simple et des facteurs de non linéarité pour une rampe. Un fichier fits pour le vignettage et pour l'offset. 

Dans le dossier *pyETC/pyETC/telescope_database* vous pouvez choisir le téléscope (associé à sa/ses caméra/s). Par exemple, pour simuler des images du détecteur Teledyne il faut choisir *colibri_teledyne.hjson* et pour simuler des images faites par la caméra de Sofradir choisissez *colibri.hjson*.

Le téléscope choisi par défault est :  [colibri_teledyne.hjson](../../pyETC/pyETC/telescope_database/colibri_teledyne.hjson)

*Remarque :* C'est à l'intérieur de ces fichiers que l'on doit placer certains coefficients importants comme la taille des pixels correspondant au détecteur, la largeur des bandes de pixels de référence ou les neufs coefficients de diaphonie interpixel.

Dans la cellule ci-dessous vous pouvez donc choisir votre type de mesure et votre téléscope en remplaçant `Dark.hjson` et `'colibri_teledyne'` par ce que vous souhaitez après avoir mis les coefficients correspondant à votre simulation.

In [ ]:
Simulation=ImageSimulator(configFile=os.getenv('ImSimpy_DIR')+'/ImSimpy/configFiles/Test_des_differents_effets/01.Sources.hjson',
                          name_telescope='colibri_teledyne')

L'objet est stocké dans la variable "Simulation". Cela peut être n'importe quoi d'autre.

Si vous n'avez pas le répertoire standard, vous devez spécifier le chemin où se trouve le fichier de configuration. Voir l'exemple dans la partie 2.

### B. Explication du code principal

*Remarque sur le fonctionnement du code* : Comme dit plus haut, c'est dans les fichiers de configuration que l'on choisit quel type d'image on veut simuler. C'est dans ces fichiers que l'on choisit la partie du ciel à oberserver, mais c'est aussi à l'aide de ces fichiers que l'on peut choisir de voir qu'une partie des instruments. Il suffit de mettre à ce moment là la bonne carte en entrée du bruit intrinsèque de l'instrument et d'afficher ou non les sources, les rayons cosmique et autres signal externe ou interne au détecteur ou au télescope.

Le **coeur du code** se trouve dans le programme *ImSimpy* (qui se trouve dans *ImSimpy\ImSimpy\ImSimpy.py*) car les différents objets appelés (`Simulation`, `Test`, `AcquisitionBruitCDS`) font parti de la classe `ImageSimulator` définie au début de ce programme. 

Le code est divisé en trois parties :
* La première contient des **fonctions de configurations** (dans l’ordre : l’initialisation, la recherche des paramètres dans les fichiers de configuration, la création des en-têtes fits puis la lecture et la génération des sources et des PSF (étalement)). Cette partie s’étend donc de la fonction `__init__` à la fonction `generateFinemaps`).
* La deuxième contient toutes les fonctions qui **simulent** les différents effets (ajout des sources, non linéarité, discrétisation, …).
* La dernière partie est constitué de deux fonctions, Simulate et Rampsimulation. Comme leur nom l’indique c’est elles qui appellent les différentes fonctions de configuration puis les différentes fonctions produisant les effets dans l’ordre convenu.



# 2. Simulation d'une image

L'image est enregistrée dans le dossier *ImSimpy/images* et vous devez placez le nom ci-dessous :

In [ ]:
#File name
Simulation.nom = 'test'

#Nom du dossier dans lequel il sera enregistrée (le dossier doit être déjà existant)
Simulation.output_dir= 'Simulation d_une image'

#Read the configfile
Simulation.acquisition='simple'
Simulation.readConfigs()

#execute it:
Simulation.simulate()

*NB: Parmi les fichiers fits que vous appliquez à l'image l'offset, le gain de conversion et la cosmétique sont appliqués aux pixels de références, tandis que le bruit thermique de l'instrument et le vignetting ne leur sont pas appliqués.*

Pour récupérer le nom de l'image simulée :

In [ ]:
fits_filename=os.getenv('ImSimpy_DIR')+'/ImSimpy/images/'+Simulation.information['output']
print (fits_filename)

# 3. Simulation d'une rampe

### A. Simulation

Pour simuler une rampe choisissez dans un premier temps un type de mesure et un téléscope (cf 1ère partie). Remplacez ci-dessous dans la variable `Simulation` le type et le téléscope choisis.

Ensuite, dans la variable `bands` placez un tableau avec les bandes spectrales que vous souhaitez du système photométrique. Voici un tableau comportant toutes les bandes (par défault seule la bande J est choisie) : 
        
        bands= ['g','r','i','z','y','J','H'] 
Dans la variable `output_dir` placez le nom du dossier dans lequel la rampe sera enregistrée dans le dossier *images* de ImSimpy (vous devez créer le dossier auparavant).

![title](../ImSimpy/data/Drop-Read-Group_blueprint.png "Schéma d'une rampe avec détecteur Teledyne")

Choisissez enfin les paramètres à l'aide du schéma ci-dessus et placez les dans les variables suivantes :
   * nombre de reset : `nbReset`
   * nombre de read : `nbRead`
   * nombre de drop : `nbDrop`
   * nombre de groupes : `nbGroup`

*NB : La fonction diginoise est supprimée, son effet est déjà simulé grâce à la fonction * `intscale` * qui discretise, elle convertit les entiers en flotants. Le shotnoise non plus, la statistique est ajoutée au fur et à mesure sur chaque fonction. Et pour le moment les rayons cosmique et la persistance ne sont pas simulés car ils sont encore en recherche (il manque des mesures faites à l'aide de téléscope pour simuler les rayons cosmique).*

In [3]:
from ImSimpy.ImSimpy import ImageSimulator

Simulation=ImageSimulator(configFile=os.getenv('ImSimpy_DIR')+'/ImSimpy/configFiles/Make_one_image_is.hjson',
                          name_telescope='colibri_teledyne')

Simulation.bands = ['J']

Simulation.output_dir='H2RGSim'

Simulation.nbReset=5
Simulation.nbRead=15
Simulation.nbDrop=0
Simulation.nbGroup=1

Simulation.acquisition='ramp'
Simulation.readConfigs()
Simulation.config['verbose'] = 'False'

for band in Simulation.bands:
    
    # Select appropriate channel for the selected filter band
    if band in ['g','r','i']:
        Simulation.config['channel']='DDRAGO-B'    
    elif band in ['z','y']:
        Simulation.config['channel']='DDRAGO-R'
    elif band in['J','H']:
        Simulation.config['channel']='CAGIRE'
        
    # Set filter band
    Simulation.config['filter_band']=band
    
    # Simulation
    Simulation.Rampsimulation('data')


Setting the random number generator seed: current time
Read config file and execute ETC
Building image: /H2RGSim/Reset.fits:
	GENERATE OBJECTS CATALOG


	GENERATE PSF
PSF convolution
done
	ADD OBJECTS
Total number of objects in the input catalog = 16237
16237/16237 objects were placed on the detector
	Apply Shot noise
	Add dark current
	Add Sky background
	Add Readout Noise
	Add Digitisation Noise
	electrons2adu
	Apply Interpixel Crosstalk
	Add offset
	Discretise
	Write outputs


AttributeError: 'ImageSimulator' object has no attribute 'nbImages'

### B. Test des différents effets 

Si vous souhaitez tester le fonctionnement d'un des effets appliqué, le code ci-dessous créé ces images dans le dossier *ImSimpy\ImSimpy\images\Test_des_différents_effets* :
* Image 1 : Seulement les sources
* Image 2 : Seulement le fond du ciel
* Image 3 : Sources + fond du ciel
* Image 4 : Seulement les rayons cosmiques
* Image 5 : Sources + fond du ciel + rayons cosmiques
* etc...

L'offset n'est pas pris en compte car il empêche une bonne visibilité des effets. Le diginoise non plus car il n'a pas sa place ici, son effet est déjà simulé grâce à la fonction `intscale` qui converti les entiers en flotants.

*NB : Pour le moment les rayons cosmique et la persistance ne sont pas simulés car c'est encore en recherche !*

In [ ]:
# Première image, seulement les sources :
Test=ImageSimulator(configFile=os.getenv('ImSimpy_DIR')+'\\ImSimpy\\configFiles\\Test_des_differents_effets\\01.Sources.hjson',
                          name_telescope='colibri_teledyne')
Test.output_dir='Test_des_differents_effets'
Test.nom = '01.Sources'
Simulation.acquisition='ramp'
Test.readConfigs()
Test.simulate()

# Seulement le fond du ciel:
Test=ImageSimulator(configFile=os.getenv('ImSimpy_DIR')+'\\ImSimpy\\configFiles\\Test_des_differents_effets\\02.Sky Background.hjson',
                          name_telescope='colibri_teledyne')
Test.output_dir='Test_des_differents_effets'
Test.nom = '02.Sky Background'
Test.readConfigs()
Test.simulate()

# Sources et fond du ciel :
Test=ImageSimulator(configFile=os.getenv('ImSimpy_DIR')+'\\ImSimpy\\configFiles\\Test_des_differents_effets\\03.Sources+SkyB.hjson',
                          name_telescope='colibri_teledyne')
Test.output_dir='Test_des_differents_effets'
Test.nom = '03.Sources+SkyB'
Test.readConfigs()
Test.simulate()

# Seulement les rayons cosmiques :
Test=ImageSimulator(configFile=os.getenv('ImSimpy_DIR')+'\\ImSimpy\\configFiles\\Test_des_differents_effets\\04.CosmicRay.hjson',
                          name_telescope='colibri_teledyne')
Test.output_dir='Test_des_differents_effets'
Test.nom = '04.CosmicRay'
Test.readConfigs()
Test.simulate()

# Sources+SkyB+CosmicR :
Test=ImageSimulator(configFile=os.getenv('ImSimpy_DIR')+'\\ImSimpy\\configFiles\\Test_des_differents_effets\\05.Sources+SkyB+CosmicR.hjson',
                          name_telescope='colibri_teledyne')
Test.output_dir='Test_des_differents_effets'
Test.nom = '05.Sources+SkyB+CosmicR'
Test.readConfigs()
Test.simulate()

# Seulement la persistance :
Test=ImageSimulator(configFile=os.getenv('ImSimpy_DIR')+'\\ImSimpy\\configFiles\\Test_des_differents_effets\\06.Persistance.hjson',
                          name_telescope='colibri_teledyne')
Test.output_dir='Test_des_differents_effets'
Test.nom = '06.Persistance'
Test.readConfigs()
Test.simulate()

# Sources+SkyB+CosmicR+Persis :
Test=ImageSimulator(configFile=os.getenv('ImSimpy_DIR')+'\\ImSimpy\\configFiles\\Test_des_differents_effets\\07.Sources+SkyB+CosmicR+Persis.hjson',
                          name_telescope='colibri_teledyne')
Test.output_dir='Test_des_differents_effets'
Test.nom = '07.Sources+SkyB+CosmicR+Persis'
Test.readConfigs()
Test.simulate()

# Seulement le vignetting :
Test=ImageSimulator(configFile=os.getenv('ImSimpy_DIR')+'\\ImSimpy\\configFiles\\Test_des_differents_effets\\08.Vignetting.hjson',
                          name_telescope='colibri_teledyne')
Test.output_dir='Test_des_differents_effets'
Test.nom = '08.Vignetting'
Test.readConfigs()
Test.simulate()

# Sources+SkyB+CosmicR+Persis+Vignet :
Test=ImageSimulator(configFile=os.getenv('ImSimpy_DIR')+'\\ImSimpy\\configFiles\\Test_des_differents_effets\\09.Sources+SkyB+CosmicR+Persis+Vignet.hjson',
                          name_telescope='colibri_teledyne')
Test.output_dir='Test_des_differents_effets'
Test.nom = '09.Sources+SkyB+CosmicR+Persis+Vignet'
Test.readConfigs()
Test.simulate()

# Seulement le bruit thermique de l'instrument :
Test=ImageSimulator(configFile=os.getenv('ImSimpy_DIR')+'\\ImSimpy\\configFiles\\Test_des_differents_effets\\10.Intrument intrinsic noise.hjson',
                          name_telescope='colibri_teledyne')
Test.output_dir='Test_des_differents_effets'
Test.nom = '10.Intrument intrinsic noise'
Test.readConfigs()
Test.simulate()

# Sources+SkyB+CosmicR+Persis+Vignet+InstrumNoise :
Test=ImageSimulator(configFile=os.getenv('ImSimpy_DIR')+'\\ImSimpy\\configFiles\\Test_des_differents_effets\\11.Sources+SkyB+CosmicR+Persis+Vignet+InstrumNoise.hjson',
                          name_telescope='colibri_teledyne')
Test.output_dir='Test_des_differents_effets'
Test.nom = '11.Sources+SkyB+CosmicR+Persis+Vignet+InstrumNoise'
Test.readConfigs()
Test.simulate()

# Seulement le dark current :
Test=ImageSimulator(configFile=os.getenv('ImSimpy_DIR')+'\\ImSimpy\\configFiles\\Test_des_differents_effets\\12.Dark Current.hjson',
                          name_telescope='colibri_teledyne')
Test.output_dir='Test_des_differents_effets'
Test.nom = '12.Dark Current'
Test.readConfigs()
Test.simulate()

# Sources+SkyB+CosmicR+Persis+Vignet+InstrumNoise+DC :
Test=ImageSimulator(configFile=os.getenv('ImSimpy_DIR')+'\\ImSimpy\\configFiles\\Test_des_differents_effets\\13.Sources+SkyB+CosmicR+Persis+Vignet+InstrumNoise+DC.hjson',
                          name_telescope='colibri_teledyne')
Test.output_dir='Test_des_differents_effets'
Test.nom = '13.Sources+SkyB+CosmicR+Persis+Vignet+InstrumNoise+DC'
Test.readConfigs()
Test.simulate()

# Seulement la diaphonie interpixel :
Test=ImageSimulator(configFile=os.getenv('ImSimpy_DIR')+'\\ImSimpy\\configFiles\\Test_des_differents_effets\\14.Interpixel CrossTalk.hjson',
                          name_telescope='colibri_teledyne')
Test.output_dir='Test_des_differents_effets'
Test.nom = '14.Interpixel CrossTalk'
Test.readConfigs()
Test.simulate()

# Sources+SkyB+CosmicR+Persis+Vignet+InstrumNoise+DC+ICT :
Test=ImageSimulator(configFile=os.getenv('ImSimpy_DIR')+'\\ImSimpy\\configFiles\\Test_des_differents_effets\\15.Sources+SkyB+CosmicR+Persis+Vignet+InstrumNoise+DC+ICT.hjson',
                          name_telescope='colibri_teledyne')
Test.output_dir='Test_des_differents_effets'
Test.nom = '15.Sources+SkyB+CosmicR+Persis+Vignet+InstrumNoise+DC+ICT'
Test.readConfigs()
Test.simulate()

# Seulement la statistique, le shot noise :
Test=ImageSimulator(configFile=os.getenv('ImSimpy_DIR')+'\\ImSimpy\\configFiles\\Test_des_differents_effets\\16.Shot Noise.hjson',
                          name_telescope='colibri_teledyne')
Test.output_dir='Test_des_differents_effets'
Test.nom = '16.Shot Noise'
Test.readConfigs()
Test.simulate()

# Sources+SkyB+CosmicR+Persis+Vignet+InstrumNoise+DC+ICT+ShotNoise :
Test=ImageSimulator(configFile=os.getenv('ImSimpy_DIR')+'\\ImSimpy\\configFiles\\Test_des_differents_effets\\17.Sources+SkyB+CosmicR+Persis+Vignet+InstrumNoise+DC+ICT+ShotNoise.hjson',
                          name_telescope='colibri_teledyne')
Test.output_dir='Test_des_differents_effets'
Test.nom = '17.Sources+SkyB+CosmicR+Persis+Vignet+InstrumNoise+DC+ICT+ShotNoise'
Test.readConfigs()
Test.simulate()

# Seulement la cosmétique :
Test=ImageSimulator(configFile=os.getenv('ImSimpy_DIR')+'\\ImSimpy\\configFiles\\Test_des_differents_effets\\18.Cosmetique.hjson',
                          name_telescope='colibri_teledyne')
Test.output_dir='Test_des_differents_effets'
Test.nom = '18.Cosmetique'
Test.readConfigs()
Test.simulate()

# Sources+SkyB+CosmicR+Persis+Vignet+InstrumNoise+DC+ICT+ShotNoise+Cosmetic :
Test=ImageSimulator(configFile=os.getenv('ImSimpy_DIR')+'\\ImSimpy\\configFiles\\Test_des_differents_effets\\19.Sources+SkyB+CosmicR+Persis+Vignet+InstrumNoise+DC+ICT+ShotNoise+Cosmetic.hjson',
                          name_telescope='colibri_teledyne')
Test.output_dir='Test_des_differents_effets'
Test.nom = '19.Sources+SkyB+CosmicR+Persis+Vignet+InstrumNoise+DC+ICT+ShotNoise+Cosmetic'
Test.readConfigs()
Test.simulate()

# Seulement le bleeding :
Test=ImageSimulator(configFile=os.getenv('ImSimpy_DIR')+'\\ImSimpy\\configFiles\\Test_des_differents_effets\\20.Bleeding.hjson',
                          name_telescope='colibri_teledyne')
Test.output_dir='Test_des_differents_effets'
Test.nom = '20.Bleeding'
Test.readConfigs()
Test.simulate()

# Sources+SkyB+CosmicR+Persis+Vignet+InstrumNoise+DC+ICT+ShotNoise+Cosmetic+Bleeding :
Test=ImageSimulator(configFile=os.getenv('ImSimpy_DIR')+'\\ImSimpy\\configFiles\\Test_des_differents_effets\\21.Sources+SkyB+CosmicR+Persis+Vignet+InstrumNoise+DC+ICT+ShotNoise+Cosmetic+Bleeding.hjson',
                          name_telescope='colibri_teledyne')
Test.output_dir='Test_des_differents_effets'
Test.nom = '21.Sources+SkyB+CosmicR+Persis+Vignet+InstrumNoise+DC+ICT+ShotNoise+Cosmetic+Bleeding'
Test.readConfigs()
Test.simulate()

# Seulement le bruit de lecture :
Test=ImageSimulator(configFile=os.getenv('ImSimpy_DIR')+'\\ImSimpy\\configFiles\\Test_des_differents_effets\\22.ReadOutNoise.hjson',
                          name_telescope='colibri_teledyne')
Test.output_dir='Test_des_differents_effets'
Test.nom = '22.ReadOutNoise'
Test.readConfigs()
Test.simulate()

# Sources+SkyB+CosmicR+Persis+Vignet+InstrumNoise+DC+ICT+ShotNoise+Cosmetic+Bleeding+RON :
Test=ImageSimulator(configFile=os.getenv('ImSimpy_DIR')+'\\ImSimpy\\configFiles\\Test_des_differents_effets\\23.Sources+SkyB+CosmicR+Persis+Vignet+InstrumNoise+DC+ICT+ShotNoise+Cosmetic+Bleeding+RON.hjson',
                          name_telescope='colibri_teledyne')
Test.output_dir='Test_des_differents_effets'
Test.nom = '23.Sources+SkyB+CosmicR+Persis+Vignet+InstrumNoise+DC+ICT+ShotNoise+Cosmetic+Bleeding+RON'
Test.readConfigs()
Test.simulate()

# Seulement le gain de conversion :
Test=ImageSimulator(configFile=os.getenv('ImSimpy_DIR')+'\\ImSimpy\\configFiles\\Test_des_differents_effets\\24.Convert.hjson',
                          name_telescope='colibri_teledyne')
Test.output_dir='Test_des_differents_effets'
Test.nom = '24.Convert'
Test.readConfigs()
Test.simulate()

# Sources+SkyB+CosmicR+Persis+Vignet+InstrumNoise+DC+ICT+ShotNoise+Cosmetic+Bleeding+RON+Convert :
Test=ImageSimulator(configFile=os.getenv('ImSimpy_DIR')+'\\ImSimpy\\configFiles\\Test_des_differents_effets\\25.Sources+SkyB+CosmicR+Persis+Vignet+InstrumNoise+DC+ICT+ShotNoise+Cosmetic+Bleeding+RON+Convert.hjson',
                          name_telescope='colibri_teledyne')
Test.output_dir='Test_des_differents_effets'
Test.nom = '25.Sources+SkyB+CosmicR+Persis+Vignet+InstrumNoise+DC+ICT+ShotNoise+Cosmetic+Bleeding+RON+Convert'
Test.readConfigs()
Test.simulate()

# 4. Bruit CDS

L'acquisition de bruit CDS (Correlated double sampling ou échantillonnage double corrélé) est un mode où on peut mettre en avant le bruit. Comme dans la figure ci-dessous, quatre images sont acquises suivant deux rampes et une dernière image est créée où seulement le bruit apparaît. 

![title](../ImSimpy/data/SchémaCDS.png)
Pour simuler cette dernière on soustrait deux images consécutives pour en trouver l’écart-type. Or, avec des CMOS cet écart-type n’est pas représentatif de l’erreur, le gain et l’offset varient par pixel. C’est pourquoi, on compare 4 images, on calcul le l’écart-type deux à deux. Un coefficient de racine de 2 apparaît : 

![title](../ImSimpy/data/I2-I1.jpg)
![title](../ImSimpy/data/Racine2.jpg)


Cette dernière image subira aussi des traitements des pixels de références expliqués plus loins.



De la même manière que dans la partie trois, choisissez le type de téléscope, le type d'image à simuler, la fenêtre atmosphérique et le dossier dans lequel votre acquisition sera enregistrée.

In [ ]:
from ImSimpy.ImSimpy import ImageSimulator

AcquisitionBruitCDS=ImageSimulator(configFile=os.getenv('ImSimpy_DIR')+'/ImSimpy/configFiles/Dark.hjson',
                          name_telescope='colibri_teledyne')

AcquisitionBruitCDS.bands = ['J']

AcquisitionBruitCDS.output_dir='AcquisitionBruitCDS'

# Acquisition CDS (ne plus rien modifier) :
AcquisitionBruitCDS.nbReset=5
AcquisitionBruitCDS.nbRead=2
AcquisitionBruitCDS.nbDrop=0
AcquisitionBruitCDS.nbGroup=1

AcquisitionBruitCDS.acquisition='CDS'
AcquisitionBruitCDS.readConfigs()
AcquisitionBruitCDS.config['verbose'] = 'False'

for band in AcquisitionBruitCDS.bands:
    
    # Select appropriate channel for the selected filter band
    if band in ['g','r','i']:
        AcquisitionBruitCDS.config['channel']='DDRAGO-B'    
    elif band in ['z','y']:
        AcquisitionBruitCDS.config['channel']='DDRAGO-R'
    elif band in['J','H']:
        AcquisitionBruitCDS.config['channel']='CAGIRE'
        
    # Set filter band
    AcquisitionBruitCDS.config['filter_band']=band
    
    # Simulation
    AcquisitionBruitCDS.number=1
    AcquisitionBruitCDS.Rampsimulation('data')
    AcquisitionBruitCDS.number=2
    AcquisitionBruitCDS.Rampsimulation('data')